In [ ]:
#Class definition
class Scenario(object):
    """
    Define a gravitational scenario
    
    Parameters:
        central: Object in the center of the scenario, Body
        neighbor: Objects surrounding the center, tuple of bodies

    """
    def __init__(self,central,neighbors):
        self.central=central
        self.neighbors=neighbors
        
    def updateScenario(self,tdb):
        self.central.updateBody(tdb)
        for neighbor in self.neighbors:
            neighbor.updateBody(tdb)

#Unitary test
class Test(unittest.TestCase):

    tdb=Spice.str2t("2000 JAN 01 12:00:00")

    moon=Body("MOON","IAU_MOON",1*Const.SideralMonth)
    earth=Body("EARTH","ITRF93",1*Const.Day)
    ssb=Body("SSB",None)
    mes=Scenario(moon,[earth,ssb])
    
    def test_update_scenario(self):
        self.mes.updateScenario(self.tdb)
        self.assertEqual(np.isclose(self.mes.central.state,
                                    [-2.78582406e+10,1.44004083e+11,6.64975943e+07,
                                     -2.91414161e+04,-6.21310369e+03,-1.14880075e+01],
                                    rtol=1e-5).tolist(),
                          [True]*6)
        self.assertEqual(np.isclose(self.mes.central.Tbod2ecl.flatten(),
                                    [0.78422705,-0.62006192,-0.02260867,
                                     0.61987147,0.78455064,-0.01548052,
                                     0.02733653,-0.00187423,0.99962453],
                                    rtol=1e-5).tolist(),
                          [True]*9)
        self.assertEqual(np.isclose(self.earth.state,
                                    [-2.75666323e+10,1.44279062e+11,3.02263967e+07,
                                     -2.97849475e+04,-5.48211971e+03,1.84565202e-02],
                                    rtol=1e-5).tolist(),
                          [True]*6)
        self.assertEqual(np.isclose(self.earth.Tbod2ecl.flatten(),
                                    [ 1.76980593e-01,9.84214341e-01,-2.51869708e-05,
                                     -9.03007988e-01,1.62388314e-01,3.97751944e-01,
                                     3.91477257e-01,-7.03716309e-02,9.17492992e-01],
                                    rtol=1e-5).tolist(),
                          [True]*9)
    
    def timing_update_scenario(self):
        self.mes.updateScenario(self.tdb)

if __name__=='__main__':
    #Testing
    unittest.main(argv=['first-arg-is-ignored'],exit=False)
    
    #Timing
    print("Update scenario:")
    %timeit -n 1000 Test().timing_update_scenario()
    
    def propagateRay(self,tdb):

        #Update ray
        self.updateRay(tdb)
        
        #Initialize
        state=np.zeros(6)
        body=deepcopy(self.body)
        state=self.stateEcl+body.stateHelio
        et=self.tdb
        
        self.states=[]
        while body.master is not None:

            #State w.r.t. to body
            body.updateBody(et)
            state=state-body.stateHelio
            #print(f"State with respect to body {body.id}: ",state)
        
            #Get object-centric elements
            q,e,i,Omega,omega,Mo,et,mu=spy.oscelt(state,et,body.mu)
            
            #Collision
            if e<1:
                raise AssertionError(f"The object has collided against {body.id}")

            #Elements
            a=q/(1-e)
            n=np.sqrt(body.mu/np.abs(a)**3)

            #If Mo<0 (before periapsis) and raydir=-1 (towared the Earh) the periapsis is in the future
            etp=et+self.raydir*Mo/n
            self.states+=[(et,body.mu,[q,e,i,Omega,omega,Mo],state)]
            
            #Propagate up to the Hill-Radius of body
            rhill=body.rhill
            fd=np.arccos((q*(1+e)/rhill-1)/e)
            Hd=2*np.arctanh(np.sqrt((e-1)/(e+1))*np.tan(fd/2))
            Md=e*np.sinh(Hd)-Hd
            
            #Time until passage by the Hill-Radius
            deltat=Md/n
            
            #Update body position
            thill=et+self.raydir*deltat
            body.updateBody(thill)

            #Heliocentric state
            hillstate=spy.conics([q,e,i,Omega,omega,Mo,et,body.mu],thill)
            self.states+=[(thill,body.mu,[q,e,i,Omega,omega,Mo+self.raydir*Md],hillstate)]
            
            #Next conic
            et=thill
            state=hillstate+body.stateHelio
            body=deepcopy(self.masters[body.master])
        
        self.terminal=KeplerianOrbit(Spice.Mu["SSB"])
        self.terminal.setState(state,et)
        self.states+=[(et,body.mu,self.terminal.elements,state)]



In [ ]:
"""
        if self.elements[1]<1:
            E=np.arccos(self.derivatives[-2])
        else:
            E=np.arccosh(self.derivatives[-2])
        M=self.s*(E-self.elements[1]*self.derivatives[-1])
        #"""
        

In [ ]:
    def propagateRay(self,tdb):

        #Update ray
        self.updateRay(tdb)
        
        #Initialize
        state=np.zeros(6)
        body=deepcopy(self.body)
        state=self.stateEcl+body.stateHelio
        et=self.tdb
        
        self.states=[]
        while body.master is not None:

            #State w.r.t. to body
            body.updateBody(et)
            state=state-body.stateHelio

            #Get object-centric elements
            q,e,i,Omega,omega,Mo,et,mu=spy.oscelt(state,et,body.mu)
            self.states+=[(et,body.mu,[q,e,i,Omega,omega,Mo],state,f"Surface of {body.id}")]
       
            #Collision
            if e<1:
                raise AssertionError(f"The object has collided against {body.id}")

            #Create body-centric orbit
            orbit=KeplerianOrbit(body.mu)
            orbit.setElements([q,e,i,Omega,omega,Mo],et)
            
            #Propagate until Hill radius
            Md,deltat,state=orbit.calcStateByDistance(body.rhill,direction=self.raydir)
            orbit.elements[5]+=Md
            self.states+=[(et,body.mu,orbit.elements,state,f"Escape from {body.id}")]

            #Next conic
            body.updateBody(et+deltat)
            et=et+deltat
            state=state+body.stateHelio
            body=deepcopy(self.masters[body.master])
        
        self.terminal=KeplerianOrbit(Spice.Mu["SSB"])
        self.terminal.setState(state,et)
        self.states+=[(et,body.mu,self.terminal.elements,state,f"Final escape")]
        



In [ ]:
    
    def propagateEscapeState(self,state,body=None,et=0,direction=1):
        """
        Compute the transformation from the position w.r.t. to the central body of the ray
        to the position w.r.t. the master of the central body
        """
        q,e,i,Omega,omega,M,et,mu=spy.oscelt(state,et,body.mu)
        if e<1:
            raise AssertionError(f"The object has collided against {body.id}")        
        orbit=KeplerianOrbit(body.mu)
        orbit.setElements([q,e,i,Omega,omega,M],et)
        Md,deltat,state=orbit.calcStateByDistance(body.rhill,direction=direction)
        helio=body.calcState(et+deltat)
        statehelio=state+helio
        return Md,deltat,orbit,state,helio,statehelio
        

        

In [ ]:
    def calcJacobianDeterminant(self):
        """
        Compute Jacobian determinant
        """
            
        #Jxi := dXgeo/dRimp
        Rimp=self.Rimp
        state=self.stateBody
        Jxi,Jix=Jacobians.calcImpactJacobian(self.body,Rimp,state)
        detJxi=np.linalg.det(Jxi)
            
        #Jhx := dXgeo/dXhel
        detJhx=1.0
        
        #Jeh := dehel/dXhel
        hel_et,hel_mu,hel_elements,hel_state,hel_name=self.states[-1]
        hel=KeplerianOrbit(hel_mu)
        hel.setElements(hel_elements,0)
        Jhe=Jacobians.calcKeplerianJacobians(hel.mu,hel.celements,hel.state)
        Jeh=np.linalg.inv(Jhe)
        detJeh=np.linalg.det(Jeh)
        
        #|Jei| := |Jeh| |Jhx| |Jxi|
        detJei=detJeh*detJhx*detJxi
        
        return detJei
    


In [ ]:
    def propagateRay(self,tdb):

        #Update ray
        self.updateRay(tdb)
        
        #Set initial orbit
        et=tdb
        body=self.body
        helio=self.body.stateHelio
        state=self.stateEcl+helio
        
        self.states=[]
        while body.master is not None:

            #State with respect to body
            state-=helio
            self.states+=[(et,body.mu,None,state,helio,state+helio,f"Surface of {body.id}")]

            #Scape state
            Md,deltat,orbit,state,helio,statehelio=\
                self.propagateEscapeState(state,body=body,et=et,direction=self.raydir)
            self.states+=[(et,body.mu,orbit,state,helio,statehelio,f"Escape from {body.id}")]
            
            #Position with respect to new body
            et+=deltat
            state=statehelio
            body=self.masters[body.master]
            helio=body.calcState(et)
        
        self.terminal=KeplerianOrbit(Spice.Mu["SSB"])
        self.terminal.setState(state,et)
        self.states+=[(et,Spice.Mu["SSB"],self.terminal,state-helio,helio,statehelio,f"Final escape")]
    
    def propagateEscapeState(self,state,body=None,et=0,direction=1):
        """
        Compute the transformation from the position w.r.t. to the central body of the ray
        to the position w.r.t. the master of the central body
        """
        q,e,i,Omega,omega,M,et,mu=spy.oscelt(state,et,body.mu)
        if e<1:
            raise AssertionError(f"The object has collided against {body.id}")        
        orbit=KeplerianOrbit(body.mu)
        orbit.setElements([q,e,i,Omega,omega,M],et)
        Md,deltat,state=orbit.calcStateByDistance(body.rhill,direction=direction)
        helio=body.calcState(et+deltat)
        statehelio=state+helio
        return Md,deltat,orbit,state,helio,statehelio



In [ ]:
pool=mp.Pool(NP)
elTime(0)
R=[pool.apply_async(rayProcessingMulti,args=(initials,),callback=joinResults) for initials in cinitials]
pool.close()
pool.join()
elTime()